In [1]:
!pip install transformers

In [2]:
!pip install tqdm

In [3]:
!pip install torchsnooper

In [4]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D" -O 10layers_12heads_1024len_768embd_full_corpus_16bsize.zip && rm -rf /tmp/cookies.txt

--2020-12-07 11:13:03--  https://docs.google.com/uc?export=download&confirm=pReA&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D
Resolving docs.google.com (docs.google.com)... 172.217.25.110, 2404:6800:4004:819::200e
Connecting to docs.google.com (docs.google.com)|172.217.25.110|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-2c-docs.googleusercontent.com/docs/securesc/30hl151f4th7go9ejfjjs0mmu073mcsj/k2mq5nb6qqaj6i3dj9tp0lsj215cdah8/1607310750000/08564091608614806726/07300456246211891249Z/1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D?e=download [following]
--2020-12-07 11:13:03--  https://doc-0o-2c-docs.googleusercontent.com/docs/securesc/30hl151f4th7go9ejfjjs0mmu073mcsj/k2mq5nb6qqaj6i3dj9tp0lsj215cdah8/1607310750000/08564091608614806726/07300456246211891249Z/1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D?e=download
Resolving doc-0o-2c-docs.googleusercontent.com (doc-0o-2c-docs.googleusercontent.com)... 172.217.175.1, 2404:6800:4004:81c::2001
Connecting to doc-0o

In [5]:
import sys

In [6]:
GITHUB_REPO = "GPT2-Chinese"

In [7]:
!rm -rf {GITHUB_REPO}
!git clone https://github.com/Morizeyao/{GITHUB_REPO}.git {GITHUB_REPO}

Cloning into 'GPT2-Chinese'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 230 (delta 13), reused 18 (delta 7), pack-reused 199
Receiving objects: 100% (230/230), 13.43 MiB | 6.89 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [8]:
if not GITHUB_REPO in sys.path:
    sys.path += [GITHUB_REPO]

In [9]:
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
    sys.path += ['bertviz_repo']
!pip install regex

In [10]:
import json
import torch
import textwrap
import torchsnooper
import transformers
import torch.nn.functional as F
from tqdm import trange
from IPython.core.display import display, HTML

In [11]:
pretrained_model = '10layers_12heads_1024len_768embd_full_corpus_16bsize'

In [12]:
!unzip {pretrained_model}.zip

Archive:  10layers_12heads_1024len_768embd_full_corpus_16bsize.zip
  inflating: home/ec2-user/SageMaker/tmp/GPT2-Chinese/model/10layers_12heads_1024len_768embd_full_corpus_16bsize/config.json  
  inflating: home/ec2-user/SageMaker/tmp/GPT2-Chinese/model/10layers_12heads_1024len_768embd_full_corpus_16bsize/pytorch_model.bin  
  inflating: home/ec2-user/SageMaker/tmp/GPT2-Chinese/cache/vocab_small.txt  


In [13]:
sagemaker_base_path = 'home/ec2-user/SageMaker/tmp/GPT2-Chinese'
config_file = 'config.json'
model_ckpt = "pytorch_model.bin"
vocab_file = "vocab_small.txt"

In [14]:
!rm {config_file} {model_ckpt} {vocab_file}

In [15]:
!mv {sagemaker_base_path}/model/{pretrained_model}/{config_file} {config_file}
!mv {sagemaker_base_path}/model/{pretrained_model}/{model_ckpt} {model_ckpt}
!mv {sagemaker_base_path}/cache/{vocab_file} {vocab_file}

In [16]:
from transformers import BertTokenizer
from transformers import GPT2Config, GPT2LMHeadModel

In [17]:
config = GPT2Config.from_json_file(config_file)
config.output_attentions = True

In [18]:
model = GPT2LMHeadModel.from_pretrained(".", config=config)
tokenizer = BertTokenizer(vocab_file=vocab_file)

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(13317, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [20]:
def is_word(word):
    for item in list(word):
        if item not in 'qwertyuiopasdfghjklzxcvbnm':
            return False
    return True

In [21]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

In [22]:
def fast_sample_sequence(model,context,length,temperature=1, top_k=0, top_p=0.0,device='cpu'):
    inputs = torch.LongTensor(context).view(1, -1).to(device)
    if len(context) > 1:
        _, past = model(inputs[:, :-1], None)[:2]
        prev = inputs[:, -1].view(1, -1)
    else:
        past = None
        prev = inputs
    generate = [] + context
    with torch.no_grad():
        for i in trange(length):
            output = model(prev, past=past)
            output, past = output[:2]
            output = output[-1].squeeze(0) / temperature
            filtered_logits = top_k_top_p_filtering(output, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(torch.softmax(filtered_logits, dim=-1), num_samples=1)
            
            # redraw if [UNK]
            if next_token.unsqueeze(0) != 100:
                generate.append(next_token.item())
                prev = next_token.view(1, 1)

    return generate

In [23]:
def get_html(context, generated_text, novel_name='', algorithm=''):
    if generated_text[-1] != '。':
        if generated_text[-1] == '，':
            generated_text= generated_text[:-1]
        generated_text += ' ...'
    
    html = """
    <html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
    * {
      box-sizing: border-box;
    }

    /* Create two unequal columns that floats next to each other */
    .column {
      float: left;
      padding: 10px;
    }

    .left {
      width: 20%;
    }

    .right {
      width: 80%;
    }

    /* Clear floats after the columns */
    .row:after {
      content: "";
      display: table;
      clear: both;
    }
    </style>
    </head>
    <body>

    <div style="background-color:#FFFFF0 !important">
        <div class="row">
          <div class="column left">
            <h3 style="color: #bfbaba;text-align:center">
                GPT-2看了這個標題的內容：
                novel_name
                
            </h2>
          </div>
          <div class="column right">
            <h3 style="color: #708090;line-height: 1.5">context</h3>
          </div>
        </div>

        <hr/>

        <div class="row">
          <div class="column left">
            <h3 style="color: #00477D;text-align:center">
                algorithm
                GPT2說...
            </h2>
          </div>
          <div class="column right">
            <h3 style="color: #006374;line-height: 1.5">generated_text</h3>

          </div>
        </div>
    </div>

    </body>
    </html>


    """.replace('context', context).replace('generated_text', generated_text).replace("novel_name", f'《{novel_name}》<br/>')
    
    if not algorithm:
        html = html.replace("algorithm", "")
    else:
        html = html.replace("algorithm", f'{algorithm}<br/>')
    
    
    return html

In [24]:
def generate(context, topk, topp, temperature, device, line_len=40, novel_name=''):
    context_tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(context))
        
    # auto-regressive
    out = fast_sample_sequence(
        model=model, length=length,
        context=context_tokens,
        temperature=temperature, top_k=topk, top_p=topp, device=device
    )

    # rendering
    tokens = tokenizer.convert_ids_to_tokens(out)

    for i, item in enumerate(tokens):
        if item == '[MASK]':
            tokens[i] = ''
        if item == '[CLS]' or item == '[SEP]':
            tokens[i] = '\n'
    
    generated_text = ''.join(tokens).strip().replace(context, '')
    html = get_html(context, generated_text, novel_name=novel_name)
    
    return html, generated_text

In [25]:
#@ title 生成參數、Decoding 策略設定（生成溫度等）
nsamples = 1
batch_size = 1
length = model.config.n_ctx // 2

topk = 30
topp = 0
temperature = 1

In [26]:
# 改變此變數來選擇不同的前文脈絡。當然你也可以自行填加
sample_idx = 1

In [27]:
sampled_contexts =[
    ('寶可夢 劍／盾', '任天堂年度大作《寶可夢 劍／盾》正式發表，上市前就備受訓練家們期待，根據寶可夢公司最新公告，首週全球銷售突破  600 萬，打破 Switch 新紀錄，然而玩家社群們卻是負評如潮。'),
    ('「Cybertrunk」將於《電馭叛客 2077》登場?', '行事風格向來不按牌理出牌的特斯拉汽車執行長伊隆馬斯克 (Elon Musk) 在上週的發表會上發表了一台造型相當前衛的全新車種－「The Cybertruck」。這台充滿Cyberpunk風格的全新電動車標榜著環保的特性，以及防彈的強化玻璃，即使在發表會上出了點小包，一砸就出現裂痕。然而，即使無法購買這台真車的玩家們，也許有機會在《電馭叛客 2077》中駕駛這台獨特的汽車，奔馳於遊戲中的「夜城」(Night City) 街道。'),
]

In [28]:
sample = sampled_contexts[sample_idx]
novel_name, context = sample

In [29]:
html, generated_text = generate(context, topk, topp, temperature, device, novel_name=novel_name)

100%|██████████| 512/512 [00:15<00:00, 33.90it/s]


In [30]:
generated_text

'行事風格向來不按牌理出牌的特斯拉汽車執行長伊隆馬斯克(el##onmu##sk)在上週的發表會上發表了一台造型相當前衛的全新車種－「thec##y##bert##ru##ck」。這台充滿c##y##ber##pu##nk風格的全新電動車標榜著環保的特性，以及防彈的強化玻璃，即使在發表會上出了點小包，一砸就出現裂痕。然而，即使無法購買這台真車的玩家們，也許有機會在《電馭叛客207##7》中駕駛這台獨特的汽車，奔馳於遊戲中的「夜城」(nightcity)街道。\n楊逍本是冷冷的一乾人，但這昭宗親王的林兼具神態，文武又講究的是一位武學奇才，他說的又多又多又多似幾分匠，更其脾氣很好，見楊逍踏過之時，頗有幾分怠忽，頗得心下暗喜。他見楊逍踏過船頭，又穩穩又看船頭，待航近，料想楊逍派出巡查究起來。這艘甲板不過三十里，待秋水射到，第二次第二次再行加速，料想水笙必定躲在船梢，危急中張開雙臂，不及細想，倒躍回岸，抱住船板，這才抱住。只見船板上黑沈沈的一片，他雙手用力一推，鐵鍊當即牢牢掀住，原來抱了個空。這麼一個巨大的水手，楊過在水里沒扳動，一個踉的倒退幾下，只能僵直，這時手足酸軟，雙足卻又向下虛蕩傾，只覺腰當胸，身子一挺，竟已抱住了他空。\n那大官連聲驚呼，楊過右臂托在他的肩頭，借力借力轉舵，那大漢又摔入江中。這大漢正自滿腔忿怒的發洩，旁觀眾人無不驚訝。區區一個驚訝之下，急忙扶起，急上岸，見左邊船舷上水面坐著一對大船，原來是艘船夫楊過抓住船板，接著抱著他躍上岸來的。\n楊過一見，忙伸臂扼在楊過背心，叫道：「老爺，你沒事麼？我也不要你報仇，好！」但見天色漸暗，那大船卻又不會移動，楊過已然靠近，忙指揮孫李莫愁道：「老爺子，你給我死了，我定要殺你！」楊過「嗯」了一聲。他不肯做'

In [31]:
display(HTML(html))

## 視覺化自注意力機制

In [32]:
#@title BertViz wrapper
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
      '''))

In [33]:
from bertviz import head_view as show_head
from bertviz import model_view as show_model
from bertviz import neuron_view as show_neuron
#from transformers import BertTokenizer, BertModel
from transformers import GPT2Tokenizer, GPT2Model

In [34]:
def call_html(view):
    import IPython
    if view in ['model', 'neuron']:
        display(IPython.core.display.HTML('''
             <script src="/static/components/requirejs/require.js"></script>
             <script>
               requirejs.config({
                 paths: {
                   base: '/static/base',
                   "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min",
                   jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
                 },
               });
             </script>
        '''))
    else:
        display(IPython.core.display.HTML('''
             <script src="/static/components/requirejs/require.js"></script>
             <script>
               requirejs.config({
                 paths: {
                   base: '/static/base',
                   "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
                   jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
                 },
               });
             </script>
       '''))

In [35]:
def show(model, tokenizer, text, view, model_type='gpt2'):
    call_html(view)
    show_func = {'head': show_head, 'model': show_model, 'neuron': show_neuron}
    show_func[view](model, model_type, tokenizer, text)

In [36]:
def show_head_view(text, view, model_type='gpt2'):
    call_html(view)
    #model_version = 'gpt2'
    #model = GPT2Model.from_pretrained(model_version, output_attentions=True)
    #model.to('cpu')
    #tokenizer = GPT2Tokenizer.from_pretrained(model_version)
    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention = model(input_ids, token_type_ids=token_type_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    show_head(attention, tokens)

In [37]:
def show_model_view(text, view, model_type='gpt2'):
    call_html(view)
    #model_version = 'gpt2'
    #model = GPT2Model.from_pretrained(model_version, output_attentions=True)
    #model.to('cpu')
    #tokenizer = GPT2Tokenizer.from_pretrained(model_version)
    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention = model(input_ids, token_type_ids=token_type_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    show_model(attention, tokens)

In [38]:
#gpt2_model = GPT2Model.from_pretrained('.')
#gpt2_model.to('cpu')
#gpt2_model.eval()

In [39]:
text = '小明今天沒來，聽說他生病了'
#text = 'Mary at the store, she bought apples, oranges, bananas,'

## 視覺化

In [40]:
print(text)

小明今天沒來，聽說他生病了


In [41]:
#@title head 模式
show_head_view(text, 'head')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
#@title model 模式
show_model_view(text, 'model')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
from bertviz import head_view
from transformers import GPT2Tokenizer, GPT2Model

In [44]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [45]:
model_version = 'gpt2'
model = GPT2Model.from_pretrained(model_version, output_attentions=True)
tokenizer = GPT2Tokenizer.from_pretrained(model_version)

#text = "The quick brown fox jumps over the lazy dogs."
inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
input_ids = inputs['input_ids']
attention = model(input_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()
head_view(attention, tokens)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [ ]:
model_version = 'gpt2'
model = GPT2Model.from_pretrained(model_version, output_attentions=True)
tokenizer = GPT2Tokenizer.from_pretrained(model_version)
#text = "The quick brown fox jumps over the lazy dogs."
inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
input_ids = inputs['input_ids']
attention = model(input_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
head_view(attention, tokens)